# Implementing a Custom Hash Table with Collision Handling

In [ ]:
import random
import string
import time

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class HashTableChaining:
    def __init__(self, size=100):
        self.size = size
        self.table = [None] * self.size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        idx = self._hash(key)
        head = self.table[idx]
        curr = head
        while curr:
            if curr.key == key:
                curr.value = value
                return
            curr = curr.next
        new_node = Node(key, value)
        new_node.next = head
        self.table[idx] = new_node

    def get(self, key):
        idx = self._hash(key)
        curr = self.table[idx]
        while curr:
            if curr.key == key:
                return curr.value
            curr = curr.next
        return None

    def delete(self, key):
        idx = self._hash(key)
        curr = self.table[idx]
        prev = None
        while curr:
            if curr.key == key:
                if prev:
                    prev.next = curr.next
                else:
                    self.table[idx] = curr.next
                return True
            prev = curr
            curr = curr.next
        return False

    def display(self):
        for i, node in enumerate(self.table):
            print(f"Bucket {i}:", end=" ")
            curr = node
            while curr:
                print(f"({curr.key}: {curr.value})", end=" -> ")
                curr = curr.next
            print("None")

class HashTableOpenAddressing:
    def __init__(self, size=100):
        self.size = size
        self.table = [None] * self.size
        self.deleted = object()

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        idx = self._hash(key)
        for i in range(self.size):
            probe = (idx + i) % self.size
            if self.table[probe] is None or self.table[probe] is self.deleted:
                self.table[probe] = (key, value)
                return
            elif self.table[probe][0] == key:
                self.table[probe] = (key, value)
                return
        raise Exception("Hash table is full")

    def get(self, key):
        idx = self._hash(key)
        for i in range(self.size):
            probe = (idx + i) % self.size
            if self.table[probe] is None:
                return None
            elif self.table[probe] is self.deleted:
                continue
            elif self.table[probe][0] == key:
                return self.table[probe][1]
        return None

    def delete(self, key):
        idx = self._hash(key)
        for i in range(self.size):
            probe = (idx + i) % self.size
            if self.table[probe] is None:
                return False
            elif self.table[probe] is self.deleted:
                continue
            elif self.table[probe][0] == key:
                self.table[probe] = self.deleted
                return True
        return False

    def display(self):
        for i, item in enumerate(self.table):
            if item is None:
                print(f"Slot {i}: Empty")
            elif item is self.deleted:
                print(f"Slot {i}: Deleted")
            else:
                print(f"Slot {i}: ({item[0]}: {item[1]})")

# Helper functions for testing and benchmarking
def random_string(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def benchmark(table_class, n=1000, size=2003):
    keys = [random_string() for _ in range(n)]
    values = [random.randint(1, 10000) for _ in range(n)]
    table = table_class(size=size)
    # Insert
    start = time.time()
    for k, v in zip(keys, values):
        table.insert(k, v)
    insert_time = time.time() - start
    # Search
    start = time.time()
    for k in keys:
        _ = table.get(k)
    search_time = time.time() - start
    return insert_time, search_time

# Test cases
def test_hash_tables():
    print("Testing Chaining...")
    ht_chain = HashTableChaining(size=10)
    ht_chain.insert("a", 1)
    ht_chain.insert("b", 2)
    assert ht_chain.get("a") == 1
    assert ht_chain.get("b") == 2
    ht_chain.delete("a")
    assert ht_chain.get("a") is None

    print("Testing Open Addressing...")
    ht_open = HashTableOpenAddressing(size=10)
    ht_open.insert("a", 1)
    ht_open.insert("b", 2)
    assert ht_open.get("a") == 1
    assert ht_open.get("b") == 2
    ht_open.delete("a")
    assert ht_open.get("a") is None
    print("All tests passed.")

# Run tests
test_hash_tables()

# Benchmark
print("\nBenchmarking with 1000 random key-value pairs...")
chain_insert, chain_search = benchmark(HashTableChaining)
open_insert, open_search = benchmark(HashTableOpenAddressing)
print(f"Chaining: Insert time = {chain_insert:.4f}s, Search time = {chain_search:.4f}s")
print(f"Open Addressing: Insert time = {open_insert:.4f}s, Search time = {open_search:.4f}s")